# Базовая валидация схемы данных

В этом ноутбуке мы рассмотрим основные возможности валидации данных с использованием схемы.

## Подготовка окружения

In [ ]:
import sys
sys.path.append('../../')

import pandas as pd
import numpy as np
from core.data.schema import DataSchema, ColumnSchema, DataType
from core.data.validation import SchemaValidator

## Создание схемы данных

In [ ]:
# Создание схемы данных
schema = DataSchema([
    ColumnSchema(
        name='id',
        data_type=DataType.INTEGER,
        required=True,
        constraints={
            'unique': True,
            'min': 1
        }
    ),
    ColumnSchema(
        name='name',
        data_type=DataType.STRING,
        required=True,
        constraints={
            'pattern': r'^[A-Za-z\s]+$'
        }
    ),
    ColumnSchema(
        name='age',
        data_type=DataType.INTEGER,
        required=True,
        constraints={
            'min': 0,
            'max': 150
        }
    ),
    ColumnSchema(
        name='email',
        data_type=DataType.STRING,
        required=True,
        constraints={
            'pattern': r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        }
    ),
    ColumnSchema(
        name='status',
        data_type=DataType.CATEGORICAL,
        required=True,
        constraints={
            'categories': ['active', 'inactive', 'pending']
        }
    )
])

## Подготовка тестовых данных

In [ ]:
# Создание корректных данных
valid_data = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['John Doe', 'Jane Smith', 'Bob Wilson'],
    'age': [25, 30, 35],
    'email': ['john@example.com', 'jane@example.com', 'bob@example.com'],
    'status': ['active', 'inactive', 'pending']
})

# Создание некорректных данных
invalid_data = pd.DataFrame({
    'id': [1, 1, 3],  # Дублирующийся id
    'name': ['John123', 'Jane Smith', 'Bob Wilson'],  # Имя с цифрами
    'age': [-1, 30, 200],  # Некорректный возраст
    'email': ['invalid-email', 'jane@example.com', 'bob@example.com'],  # Некорректный email
    'status': ['active', 'invalid', 'pending']  # Некорректный статус
})

## Создание и использование валидатора

In [ ]:
# Создание валидатора
validator = SchemaValidator(schema)

# Валидация корректных данных
try:
    validator.validate(valid_data)
    print("Данные успешно валидированы")
except ValueError as e:
    print(f"Ошибка валидации: {e}")

# Валидация некорректных данных
try:
    validator.validate(invalid_data)
except ValueError as e:
    print(f"Ошибка валидации: {e}")

## Валидация отдельных колонок

In [ ]:
# Проверка email
try:
    validator._validate_string(invalid_data['email'], schema.get_column_info('email'))
except ValueError as e:
    print(f"Ошибка валидации email: {e}")

# Проверка возраста
try:
    validator._validate_integer(invalid_data['age'], schema.get_column_info('age'))
except ValueError as e:
    print(f"Ошибка валидации возраста: {e}")

# Проверка статуса
try:
    validator._validate_categorical(invalid_data['status'], schema.get_column_info('status'))
except ValueError as e:
    print(f"Ошибка валидации статуса: {e}")